In [3]:
from scipy.special import comb

In [31]:
def nkcount(n, k, bmin=1, bmax=None, verbose=False):
    
    # n = number of objects
    # k = number of bins
    # bmin = minimum occupancy of a bin
    # bmax = maximum occupancy of a bin
    #
    # returns 
    # (1) number of combinations without upper limit on bin occupancy
    # (2) correction due to upper limit on bin occupancy
    # (3) number of combinations with upper limit on bin occupancy
    
    # Handle edge cases where any of the inputs are negative
    if (n<0 or k<0 or bmin<0 or bmax<0):
        return(0, 0, 0)
    
    # Make adjustments to calculations based on bmin
    # (1) If bmin = 0, calculate assignment of n objects to k non-empty bins
    #
    # (2) If bmin = 1, calculate assignment of n objects to k possibly empty bins
    #
    # (3) If bmin > 1, shift n and bmax to account for higher minimum occupancy
    #     then calculate assignment of n objects to k non-empty bins 
    
    if bmin == 0:
        unconstrained = comb(n-1+k, k-1, exact=True)
        empty_bins = True
    elif bmin == 1:
        unconstrained = comb(n-1, k-1, exact=True)
        empty_bins = False
    else:
        n -= (bmin-1) * k
        bmax -= (bmin-1)
        unconstrained = comb(n-1, k-1, exact=True)
        empty_bins = False
    
    # Return early if any of the following conditions are met
    # (1) No constraints on bmax (bmax >= n or bmax not set)
    # (2) Number of elements too large subject to constraints (bmax*k < n)
    # (3) All bins must have same occupancy (k*bmin = n)
    
    if bmax is None:
        return(unconstrained, 0, unconstrained)
    elif (bmax*k < n):
        return(0, 0, 0)
    elif (bmin*k == n):
        return(1, 0, 1)

    # If we made it to here, need to calculate correction base on
    # principle of inclusion and exclusion (PIE)     
    
    correction = 0
    for nlarge in range(bmax+1, n+1): # Double check limits
        nsmall = n - nlarge
        for klarge in range(1, k+1): # Double check limits
            ksmall = k - klarge
            if empty_bins:
                csmall = comb(nsmall-1+ksmall, ksmall-1, exact=True)
            else:
                csmall = comb(nsmall-1, ksmall-1, exact=True)
            clarge = comb(nlarge-klarge*bmax - 1, klarge-1, exact=True)
            cselect = comb(k, klarge, exact=True)
            sign = (-1)**klarge
            term = csmall * clarge * cselect * sign
            correction += term

    return(unconstrained, correction, unconstrained+correction)

In [21]:
print(nkcount(26, 8, 1, 6), '     expect 125588')
print(nkcount(27, 8, 1, 6), '     expect 133288')
print(nkcount(28, 8, 1, 6), '     expect 135954')
print(nkcount(29, 8, 1, 6), '   expect 133288')
print(nkcount(30, 8, 1, 6), '   expect 125588')
print(nkcount(31, 8, 1, 6), '   expect 113688')
print(nkcount(26, 8, 2, 6), '         expect 13140')
print(nkcount(27, 8, 2, 6), '        expect 18320')
print(nkcount(28, 8, 2, 6), '        expect 23940')
print(nkcount(29, 8, 2, 6), '        expect 29400')
print(nkcount(30, 8, 2, 6), '       expect 34000')
print(nkcount(31, 8, 2, 6), '      expect 37080')
print(nkcount(26, 8, 3, 6), '                   expect 36')
print(nkcount(27, 8, 3, 6), '                 expect 120')
print(nkcount(28, 8, 3, 6), '                expect 322')
print(nkcount(29, 8, 3, 6), '               expect 728')
print(nkcount(30, 8, 3, 6), '            expect 1428')
print(nkcount(31, 8, 3, 6), '            expect 2472')
print(nkcount(19, 3, 1, 6),       '                     expect 0')
print(nkcount(25, 4, 1, 6),       '                     expect 0')
print(nkcount(26, 8, 0, 6), '   expect 376160')
print(nkcount(31, 8, 0, 6), ' expect 193880')
print(nkcount(36, 8, 0, 6), '  expect 44052')
print(nkcount(36, 36, 1, 6), '                     expect 1')
print(nkcount(36, 18, 2, 6), '                     expect 1')
print(nkcount(36, 12, 3, 8), '                     expect 1')

(480700, -355112, 125588)      expect 125588
(657800, -524512, 133288)      expect 133288
(888030, -752076, 135954)      expect 135954
(1184040, -1050752, 133288)    expect 133288
(1560780, -1435192, 125588)    expect 125588
(2035800, -1922112, 113688)    expect 113688
(19448, -6308, 13140)          expect 13140
(31824, -13504, 18320)         expect 18320
(50388, -26448, 23940)         expect 23940
(77520, -48120, 29400)         expect 29400
(116280, -82280, 34000)        expect 34000
(170544, -133464, 37080)       expect 37080
(36, 0, 36)                    expect 36
(120, 0, 120)                  expect 120
(330, -8, 322)                 expect 322
(792, -64, 728)                expect 728
(1716, -288, 1428)             expect 1428
(3432, -960, 2472)             expect 2472
(0, 0, 0)                      expect 0
(0, 0, 0)                      expect 0
(4272048, -3895888, 376160)    expect 376160
(12620256, -12426376, 193880)  expect 193880
(32224114, -32180062, 44052)   expect 44052